# RDP Generic CFS Bulk File Workflow

### Importing libararies

In [1]:
import os
import sys
from dotenv import dotenv_values
config = dotenv_values(".env")

### Set RDP credentials and Initial Parameters

In [2]:
username = config['RDP_USERNAME'] #Or replace with your RDP Machine-ID
password = config['RDP_PASSWORD'] #Or replace with your RDP Password
clientId = config['RDP_APP_KEY'] #Or replace with your RDP APP Key

RDP_HOST= 'https://api.refinitiv.com'
acccess_token = None
refresh_token = None
expires_in = 0

## <a id="rdp_workflow"></a>RDP APIs Application Workflow

### Step 1: Authentication with RDP APIs

Refinitiv Data Platform entitlement check is based on OAuth 2.0 specification. The first step of an application workflow is to get a token from RDP Auth Service, which will allow access to the protected resource, i.e. data REST API. 

The API requires the following access credential information:
- Username: The username. 
- Password: Password associated with the username. 
- Client ID: This is also known as ```AppKey```, and it is generated using an App key Generator. This unique identifier is defined for the user or application and is deemed confidential (not shared between users). The client_id parameter can be passed in the request body or as an “Authorization” request header that is encoded as base64.
- Grant Type ```password```: This is for initial authentication request. An application does not have any token, so it requests new tokens using username/password combination.

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/token HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Content-Length: XXX

username=RDP_USERNAME
&password=RDP_PASSWORD
&client_id=RDP_APP_KEY
&grant_type=password
&takeExclusiveSignOnControl=true
&scope=trapi
```

Once the authentication success, the function gets the RDP Auth service response message and keeps the following RDP token information in the variables.
- **access_token**: The token used to invoke REST data API calls as described above. The application must keep this credential for further RDP APIs requests.
- **refresh_token**: Refresh token to be used for obtaining an updated access token before expiration. The application must keep this credential for access token renewal.
- **expires_in**: Access token validity time in seconds.

Next, after the application received the Access Token (and authorization token) from RDP Auth Service, all subsequent REST API calls will use this token to get the data. Please find more detail regarding RDP APIs workflow in the following resources:
- [RDP APIs: Introduction to the Request-Response API](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api) page.
- [RDP APIs: Authorization - All about tokens](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens) page.

In [3]:
#step 1 - get RDP Access Token from RDP

import http.client
import requests 
import json

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/token'
payload = f'grant_type=password&username={username}&client_id={clientId}&password={password}&takeExclusiveSignOnControl=True&scope=trapi'
try:
    response = requests.post(auth_url, 
                             headers = {'Content-Type':'application/x-www-form-urlencoded'}, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Authentication success')
    access_token = response.json()['access_token']
    refresh_token = response.json()['refresh_token']
    expires_in = int(response.json()['expires_in'])

if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Authentication success


## <a id="rdp_get_data"></a>Requesting Data from RDP APIs

That brings us to requesting the RDP APIs data. All subsequent REST API calls use the Access Token via the *Authorization* HTTP request message header as shown below to get the data. 
- Header: 
    * Authorization = ```Bearer <RDP Access Token>```

Please notice *the space* between the ```Bearer``` and ```RDP Access Token``` values.

The application then creates a request message in a JSON message format or URL query parameter based on the interested service and sends it as an HTTP request message to the Service Endpoint. Developers can get RDP APIs the Service Endpoint, HTTP operations, and parameters from Refinitiv Data Platform's [API Playground page](https://api.refinitiv.com/) - which is an interactive documentation site developers can access once they have a valid Refinitiv Data Platform account.

## <a id="rdp_get_esg_bulk"></a>Requesting Bulk Data

### Step 2: Listing the packageId using the Bucket Name

To request the CFS Bulk data, the first step is to send an HTTP ```GET``` request to the RDP ```/file-store/v1/file-sets?bucket={bucket-name}``` endpoint to list all FileSets under the input ```bucket-name```.

The HTTP Request structure is as follows:

```HTTP
GET /file-store/v1/file-sets?bucket={bucket-name}, HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Acces Token>
```

The example bucket names for RDP content set are as follows:

|              Content             |                  Bucket Name                 |           
|:--------------------------------:|:-------------------------------------------:|
| Financial Markets Reference Data | bulk-FMRD                                    | 
| Symbology                        | bulk-Symbology                               | 
| ESG                              | bulk-ESG                                     | 
| ESG - Point in Time              | bulk-ESG                                     | 
| Tick History                     | TICKHISTORY_VBD_NO_EMBARGO                   | 
| Green Revenue                    | bulk-GreenRevenue                            | 
| Starmine                         | STARMINE_PREDICTIVE_ANALYTICS_SMARTECON_LIVE | 

**Note**: The bucket name is *case-insensitive*.

Next, set a bucket name to a ```bucket_name``` variable below like to following statement:

``` Python
bucket_name = 'bulk-Symbology'
```

This notebook uses bulk-ESG as an example.

In [20]:
# set Bucket Name, this notebook use bulk-ESG as an example

bucket_name = 'bulk-ESG'

In [5]:
#step 2 - list Package IDs from bucket name

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket={bucket_name}'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive list Package IDs from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive list Package IDs from RDP APIs


Example of the first entry of package IDs, the pacakgeId is the ```packageId``` field.

In [6]:
print(json.dumps(response.json()['value'][0], sort_keys=True, indent=2, separators=(',', ':')))

{
  "attributes":[
    {
      "name":"ContentType",
      "value":"ESG Raw Current A"
    }
  ],
  "availableFrom":"2023-11-12T17:35:23Z",
  "availableTo":"2023-11-26T17:35:22Z",
  "bucketName":"bulk-ESG",
  "contentFrom":"1970-01-01T00:00:00Z",
  "contentTo":"2023-11-12T17:25:00Z",
  "created":"2023-11-12T17:35:23Z",
  "files":[
    "419e-166b-2e12c84c-8128-290f58cb7e63",
    "4b13-dc23-b0c4d35d-b773-91d689cdeda1"
  ],
  "id":"4001-cb58-8e0d697b-afdf-88851556a809",
  "modified":"2023-11-12T17:41:00Z",
  "name":"Bulk-ESG-Global-Raw-Current-SchemeA-v1-Jsonl-Init-2023-11-12T17:31:48.704Z",
  "numFiles":2,
  "packageId":"48cc-0e39-8b0a96ca-b9e1-8639999094a8",
  "status":"READY"
}


The next step is choosing the package Id. The example bucket names for RDP content set are as follows:

|              Content             |                  Bucket Name                 |              Example of Package ID             |
|:--------------------------------:|:-------------------------------------------:|:---------------------------------------------:|
| Financial Markets Reference Data | bulk-FMRD                                    | 4d48-d7ff-edcc3d38-8243-a4f7517962b8           |
| Symbology                        | bulk-Symbology                               | 4c80-73a0-fcef949b-bfde-2b9b8117cfb0           |
| ESG                              | bulk-ESG                                     | 4288-ebb6-93372235-acb2-89882a826af1           |
| ESG - Point in Time              | bulk-ESG                                     | 4173-aec7-8a0b0ac9-96f9-48e83ddbd2ad           |
| Tick History                     | TICKHISTORY_VBD_NO_EMBARGO                   | 4c01-ab9e-db594a31-a8f5-5b7852ec4638           |
| Green Revenue                    | bulk-GreenRevenue                            | Summary: 4e94-6d63-fea034dc-90e2-de33895bd4e9  |
| Green Revenue                    | bulk-GreenRevenue                            | Standard: 4316-d43b-81c40763-8e6a-0dbec8162ab1 |
| Starmine                         | STARMINE_PREDICTIVE_ANALYTICS_SMARTECON_LIVE | 40d4-1404-58533484-afe8-718650a4e0d4         

Next,set a packageId to a ```packageId``` variable below like to following statement:

``` Python
packageId = '4c80-73a0-fcef949b-bfde-2b9b8117cfb0'
```

This notebook uses 4288-ebb6-93372235-acb2-89882a826af1 packageId as an example.|       |

In [8]:
# pick the packageId you need and set to the packageId variable

#packageId = response.json()['value'][0]['packageId']
packageId = '4288-ebb6-93372235-acb2-89882a826af1'

### Step 3: Listing the Filesets of the Bulk ESG Data with the packageId

The next step is calling the CFS API with the buket name and package Id to list all FileSets using **the package Id**.

API endpint is ```/file-store/v1/file-sets?bucket={bucket-name}&packageId={packageId}```

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/file-sets?bucket={bucket-name}&packageId={packageId} HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

In [9]:
#step 3 - get file id from bucket name

CFS_url = f'{RDP_HOST}/file-store/v1/file-sets?bucket={bucket_name}&packageId={packageId}'

try:
    response = requests.get(CFS_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive FileSets list from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive FileSets list from RDP APIs


In [10]:
print(json.dumps(response.json()['value'][0], sort_keys=True, indent=2, separators=(',', ':')))

{
  "attributes":[
    {
      "name":"ContentType",
      "value":"ESG Raw Full B"
    }
  ],
  "availableFrom":"2023-11-12T17:17:33Z",
  "availableTo":"2023-11-26T17:17:32Z",
  "bucketName":"bulk-ESG",
  "contentFrom":"1970-01-01T00:00:00Z",
  "contentTo":"2023-11-12T16:05:00Z",
  "created":"2023-11-12T17:17:33Z",
  "files":[
    "4544-874e-9da0efa0-8051-c734a79d5c61",
    "4c35-1775-c1a590ea-8376-ac6c1546b908"
  ],
  "id":"401f-b3a2-1650edf4-ae9c-e65ea076e128",
  "modified":"2023-11-12T17:40:28Z",
  "name":"Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Jsonl-Init-2023-11-12T16:11:09.024Z",
  "numFiles":2,
  "packageId":"4288-ebb6-93372235-acb2-89882a826af1",
  "status":"READY"
}


The File ID is in the ```files``` array

In [11]:
# try just one file
file_id = response.json()['value'][0]['files'][1]
file_id

'4c35-1775-c1a590ea-8376-ac6c1546b908'

### Step 4: Get the Bulk file URL on AWS S3

The last step is downloading the FIle using File ID with the RDP ```/file-store/v1/files/{file ID}/stream``` endpoint.

The HTTP Request structure is as follows:

``` HTTP
GET /file-store/v1/files/{fileId}/stream?doNotRedirect=true HTTP/1.1
Host: api.refinitiv.com
Authorization: Bearer <Access Token>
```

In [12]:
#step 3 - get file stream (content) from file id

FileID_url = f'{RDP_HOST}/file-store/v1/files/{file_id}/stream?doNotRedirect=true'

try:
    response = requests.get(FileID_url, headers={'Authorization': f'Bearer {access_token}'})
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File URL from RDP APIs')
else:
    print(f'RDP APIs: CFS request failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File URL from RDP APIs


The File URL is in the ```url``` attribute.

In [13]:
file_url = response.json()['url']
file_url

'https://a206464-bulk-esg.s3.amazonaws.com/Bulk-ESG-Global-Raw-Full-SchemeB-v1/2023/11/12/Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Init-2023-11-12T16%3A11%3A09.024Z-part0.jsonl.gz?x-request-Id=f1acbb88-a85c-4ad2-95d5-48a20b774fd6&x-package-id=4288-ebb6-93372235-acb2-89882a826af1&x-client-app-id=b4842f3904fb4a1fa18234796368799086c63541&x-file-name=Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Init-2023-11-12T16%3A11%3A09.024Z-part0.jsonl.gz&x-fileset-id=401f-b3a2-1650edf4-ae9c-e65ea076e128&x-bucket-name=bulk-ESG&x-uuid=GESG1-178570&x-file-Id=4c35-1775-c1a590ea-8376-ac6c1546b908&x-fileset-name=Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Jsonl-Init-2023-11-12T16%3A11%3A09.024Z&x-event-external-name=cfs-claimCheck-download&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEOn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQCLBeDWjpDLBL3JDJNeGkbKWbdsIFxtNlAfR6n2aCMLeAIgHXdlyuG9WPIpGySOcsl82UmRLELracnxxFBbBvtJCzQqmgIIQhAEGgw2NDIxNTcxODEzMjYiDJKbrz%2Fv47u856n3iCr3AcOtX7SgriBm8XZK%2FObxziVADHu5Mm6G56tmDz5mTP19N

### Step 5: Downloading the file

Based on the S3 ```file_url``` above, the actual file name is *Bulk-ESG-Global-Raw-Full-SchemeA-v1-Init-2023-10-22T20_24_19.238Z-part09.jsonl.gz*. So you need to replace the escape character ```%3A``` with ```_``` (underscore) character.

**Note**: If you cannot download the file, please wait for a while and then retry download the file from the URL. Please do not flush the download requests.

That is all for the RDP CFS Bulk File workflow.

In [18]:
#Downlaod file
import polling2

zipfilename = file_url.split("?")[0].split("/")[-1].replace("%3A","_")
print(f'Downloading File {zipfilename} ...')

def test_result(response):
    return response.status_code == 200

try:
    response = polling2.poll(lambda: requests.get(file_url), 
                            step = 10,
                            poll_forever = True,
                            check_success= test_result)
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')

if response.status_code == 200:  # HTTP Status 'OK'
    print('Receive File Successfully')
    open(zipfilename, 'wb').write(response.content)
    print(f'{zipfilename} Saved')
else:
    print(f'RDP APIs: Request file failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Receive File Successfully
Bulk-ESG-Global-Raw-Full-SchemeB-v1-Env-Init-2023-11-12T16_11_09.024Z-part0.jsonl.gz Saved


Now you get the CFS Bulk file that you can extract and read the file.

### Step 6: Refresh Token with RDP APIs

Before the session expires (based on the ```expires_in``` parameter, in seconds) , an application needs to send a Refresh Grant request message to RDP Authentication service to get a new access token before further request data from the platform.

The API requires the following access credential information:
- Refresh Token: The current Refreh Token value from the previous RDP Authentication call
- Client ID: This is also known as ```AppKey```, and it is generated using an App key Generator. This unique identifier is defined for the user or application and is deemed confidential (not shared between users). The client_id parameter can be passed in the request body or as an “Authorization” request header that is encoded as base64.
- Grant Type ```refresh_token```: This is for getting a new Access Token. 

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/token HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Content-Length: XXX

refresh_token={current_refresh_token}
&grant_type=refresh_token
&client_id=RDP_APP_KEY
```

Once the authentication success, the function gets **access_token**, **refresh_token**, and **expires_in** from the RDP Auth service response message the same as the previous RDP Authentication call. An application must keep those value for the next Refresh Token call.

#### Caution: API Limit

The RDP Authentication service has the API limit described on the [RDP APIs: Limitations and Guidelines for the RDP Authentication Service](https://developers.lseg.com/en/article-catalog/article/limitations-and-guidelines-for-the-rdp-authentication-service) article.  If the application flushes the authentication request messages (both ```password``` and ```refresh_token``` grant_type) beyond the limit, the account will be blocked by the API Gateway. 

In [4]:
#step 6 - Refreshing Token

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/token'
payload = f'grant_type=refresh_token&client_id={clientId}&refresh_token={refresh_token}'
try:
    response = requests.post(auth_url, 
                             headers = {'Content-Type':'application/x-www-form-urlencoded'}, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Refresh Token success')
    access_token = response.json()['access_token']
    refresh_token = response.json()['refresh_token']
    expires_in = int(response.json()['expires_in'])

if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Refresh Token success


### Step 7: Revoke Token to ending the session.

This revocation mechanism allows an application to invalidate its tokens if the end-user logs out, changes identity, or exits the respective application. Notifying the authorization server that the token is no longer needed allows the authorization server to clean up data associated with that token (e.g., session data) and the underlying authorization grant.

The API requires the following HTTP Header and Credential parameter information:
- Header: 
    * Authorization = ```Basic <App Key in Base64 format>```

Please notice *the space* between the ```Basic``` and ```App Key in Base64 format``` values.
- Body parameter
    * token: The current ```Access Token``` value from the previous RDP Authentication call

The HTTP request for the RDP APIs Authentication service is as follows:

``` HTTP
POST /auth/oauth2/v1/revoke HTTP/1.1
Accept: */*
Content-Type: application/x-www-form-urlencoded
Host: api.refinitiv.com:443
Authorization: Basic <App Key in Base64>
Content-Length: XXX

token={current_Access_token}
```

In [18]:
#step 6 - Revoking Token

import base64

clientId_bytes = clientId.encode('ascii')
base64_bytes = base64.b64encode(clientId_bytes)
clientId_base64 = base64_bytes.decode('ascii')

# Send HTTP Request
auth_url = f'{RDP_HOST}/auth/oauth2/v1/revoke'
payload = f'token={access_token}'
try:
    response = requests.post(auth_url, 
                             headers = {
                                 'Content-Type':'application/x-www-form-urlencoded',
                                 'Authorization': f'Basic {clientId_base64}'
                             }, 
                             data = payload, 
                             auth = (clientId, '')
                )
except requests.exceptions.RequestException as exp:
    print(f'Caught exception: {exp}')


if response.status_code == 200:  # HTTP Status 'OK'
    print('Revoke Token success')
if response.status_code != 200:
    print(f'RDP authentication failure: {response.status_code} {response.reason}')
    print(f'Text: {response.text}')

Revoke Token success


That is all for the RDP CFS Bulk file workflow.